In [ ]:
import json
import pandas as pd
import math
import datetime

%matplotlib inline

In [ ]:
# reading the JSON data using json.load()
file = './data/GoogleLocation.json'
with open(file) as train_file:
    data_json = json.load(train_file)
    
df = pd.DataFrame({'raw': data_json['locations']})

In [ ]:
# Extract the base data
df['timestampMs'] = df.apply(lambda row: int(row['raw']['timestampMs']), axis=1)
df['latitudeE7'] = df.apply(lambda row: row['raw']['latitudeE7'], axis=1)
df['longitudeE7'] = df.apply(lambda row: row['raw']['longitudeE7'], axis=1)
df['accuracy'] = df.apply(lambda row: row['raw']['accuracy'], axis=1)

In [ ]:
# Derivatives
df['date'] = pd.to_datetime(df['timestampMs'], unit='ms')
df['year'] = df.apply(lambda row: row['date'].year, axis=1)
df['month'] = df.apply(lambda row: row['date'].month, axis=1)
df['day'] = df.apply(lambda row: row['date'].day, axis=1)

df['time_of_day'] = (df['timestampMs'] % (1000*60*60*24)) / (1000*60*60)
df['duration'] = df['timestampMs'].diff()*-1

In [ ]:
df.head()

In [ ]:
df_part = pd.DataFrame(df[(df['accuracy'] < 100) & (df['year']==2018) & (df['month']==7) & (df['day']==17)])
df_part['duration'] = df_part['timestampMs'].diff()*-1
df_part['sqrt_duration'] = df_part.apply(lambda row: math.sqrt(row['duration']), axis=1)

df_part.plot.scatter(x='longitudeE7', xerr=df_part['accuracy']*200, y='latitudeE7', c='time_of_day', s=df_part['sqrt_duration'], figsize=(16, 10), colormap='viridis')

In [ ]:
df_part['activity_raw'] = df_part.apply(lambda row: row['raw'].get('activity'), axis=1)

def gather_confidences(activities_raw, types):
    if activities_raw == None:
        return []
    
    found = []
    for activity in activities_raw:
        options = activity['activity']
        
        if options[0]['type'] == 'TILTING':
            continue
        
        confidence = 0
        for act_type in types:
            for act in filter(lambda act: act['type'] == act_type, options):
                confidence += act['confidence']
        found.append(confidence)
    return found

df_part['still_confidence'] = df_part.apply(lambda row: gather_confidences(row['activity_raw'], ['STILL']), axis=1)
df_part['moving_confidence'] = df_part.apply(lambda row: gather_confidences(row['activity_raw'], ['ON_FOOT', 'IN_VEHICLE', 'ON_BICYCLE']), axis=1)

In [ ]:
df_part.describe()

In [ ]:
def plot_single_day(year, month, day):
    df_part = pd.DataFrame(df[(df['accuracy'] < 100) & (df['year']==year) & (df['month']==month) & (df['day']==day)])
    df_part['duration'] = df_part['timestampMs'].diff()*-1
    df_part['sqrt_duration'] = df_part.apply(lambda row: math.sqrt(row['duration']), axis=1)

    df_part.plot.scatter(x='longitudeE7', xerr=df_part['accuracy']*200, y='latitudeE7', c='time_of_day', s=df_part['sqrt_duration'], figsize=(16, 10), colormap='viridis')

for i in range(1, 7):
    plot_single_day(2018, 7, i)